In [105]:
import os
if "/content" in os.getcwd():
  !rm -rf WHere_codes/
  !git clone https://github.com/keenranger/WHere_codes
  os.chdir("/content/WHere_codes")
  !pip install -r requirements.txt
from PDR.modules.DataLoader import *
from PDR.modules.Walker import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [106]:
file_name = "wt2"
pvloader = DataLoader("data/wtc.db", file_name)
pvloader.loader()
sensor_df = pvloader.sensor_df[["time", "accx", "accy", "accz", "gyrox", "gyroy",
                                "gyroz", "magx", "magy", "magz","rot0", "rot1", "rot2", "rot3", "game0", "game1", "game2", "game3"]]
sensor_df.head()

Data parsing... 
Data parsing done!


,time,accx,accy,accz,gyrox,gyroy,gyroz,magx,magy,magz,rot0,rot1,rot2,rot3,game0,game1,game2,game3
0,1591773481600,-0.280110,4.953391,8.544538,-0.149061,0.267391,0.002058,-2.708100,-1.196285,-36.576942,0.260240,-0.014232,-0.083803,0.961795,0.177664,0.190176,0.681861,0.683618
1,1591773481620,-0.241804,4.936632,8.669032,-0.094083,0.164155,-0.010159,-2.540849,-0.023419,-37.141472,0.259908,-0.013402,-0.083710,0.961905,0.176975,0.190423,0.682100,0.683490
2,1591773481640,-0.158011,4.946208,8.704943,0.147209,-0.254899,-0.066359,-3.149758,0.703300,-37.121323,0.259783,-0.013497,-0.084349,0.961881,0.178100,0.189974,0.681765,0.683657
3,1591773481660,-0.193922,4.943814,8.357799,0.073294,-0.092409,-0.026653,-2.079329,0.996955,-37.093529,0.260499,-0.015158,-0.085327,0.961577,0.179610,0.189585,0.680941,0.684191
4,1591773481680,-0.172375,4.979725,8.180635,-0.048268,0.107344,0.009999,-2.825143,-1.026440,-38.568024,0.260981,-0.015536,-0.085741,0.961403,0.179711,0.189869,0.680587,0.684437


In [107]:
walker = Walker()
for row in tqdm(sensor_df.itertuples(), total=len(sensor_df)):
    walker.step(row[0], row[1], row[2:5], row[5:8], row[8:11], row[11:15], row[15:19])


100%|██████████| 10333/10333 [00:27<00:00, 380.27it/s]


In [108]:

displacement_df = pdr_to_displacement(walker.pdr_df)
position_df = displacement_df.cumsum()
walker.pdr_df

,idx,length,body,nav,rot,game,fusion
0,134.0,0.65,-0.171987,-0.171987,-0.171987,-0.171987,-0.171987
1,166.0,0.65,-0.220675,-0.220675,-0.220675,-0.220675,-0.220675
2,189.0,0.65,-0.203770,-0.203770,-0.203770,-0.203770,-0.203770
3,220.0,0.65,-0.246858,-0.218884,-0.254104,-0.252909,-0.253017
4,243.0,0.65,-0.189309,-0.185615,-0.179319,-0.182866,-0.182943
...,...,...,...,...,...,...,...
381,10054.0,0.65,8.697382,-1.873382,-1.947241,-1.831013,-1.949746
382,10077.0,0.65,8.670565,-1.900839,-1.943060,-1.865737,-1.984470
383,10107.0,0.65,8.672323,-1.886373,-1.900604,-1.864029,-1.982092
384,10135.0,0.65,8.634584,-1.910880,-1.951592,-1.908998,-2.023774


In [109]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=position_df['rot_x'], 
    y=position_df['rot_y'],
    name="rot"))
fig.add_trace(go.Scatter(
    x=position_df['game_x'], 
    y=position_df['game_y'],
    name="game"))
fig.add_trace(go.Scatter(
    x=position_df['nav_x'], 
    y=position_df['nav_y'],
    name="nav"))

for heading, position in zip(["nav", "rot", "game"], [["nav_x","nav_y"],["rot_x","rot_y"],["game_x", "game_y"]]):    
    corner_position = position_df[position][walker.corner_df[heading]]
    fig.add_trace(go.Scatter(
    x=corner_position[position[0]], 
    y=corner_position[position[1]],
    name= heading + "corner",
    mode = 'markers')
    )
fig.update_layout(
    title="PDR",
    width = 800,
    height = 800,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()

In [110]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['rot']),name="rot"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['game']),name="game"))
fig.add_trace(go.Scatter(x=walker.headingcalc.heading_df['time'], y=np.rad2deg(walker.headingcalc.heading_df['nav']),name="nav"))
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()

In [111]:

fig = go.Figure()
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02,
    row_heights = [0.7, 0.3],
)
fig.add_trace(go.Scatter(x=walker.pdr_df['idx'], y=np.rad2deg(walker.pdr_df['nav']),name="nav"), row=1, col=1)
fig.add_trace(go.Scatter(x=walker.pdr_df['idx'], y=np.rad2deg(walker.pdr_df['rot']),name="rot"), row=1, col=1)
fig.add_trace(go.Scatter(x=walker.correlation_df['idx'], y=walker.correlation_df['correlation'],name="correlation"), row=2, col=1)
fig.update_layout(
    width = 800,
    height = 500
)
fig.show()

In [116]:
cor_df = walker.correlation_df
good_correlation = cor_df[cor_df['correlation'] < 1].index
bad_correlation = cor_df[cor_df['correlation'] >= 1].index

In [117]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=position_df['rot_x'], 
    y=position_df['rot_y'],
    name="rot"))
fig.add_trace(go.Scatter(
    x=position_df['nav_x'], 
    y=position_df['nav_y'],
    name="nav"))


for heading, position in zip(["nav", "rot"], [["nav_x","nav_y"],["rot_x","rot_y"]]):
    good_correlation_postion = position_df[position].loc[good_correlation]
    bad_correlation_position = position_df[position].loc[bad_correlation]
    fig.add_trace(go.Scatter(
    x=good_correlation_postion[position[0]], 
    y=good_correlation_postion[position[1]],
    name= heading + "good",
    mode = 'markers',
    marker_color = 'green')
    )
    fig.add_trace(go.Scatter(
    x=bad_correlation_position[position[0]], 
    y=bad_correlation_position[position[1]],
    name= heading + "bad",
    mode = 'markers',
    marker_color = 'red')
    )
fig.update_layout(
    title="PDR",
    width = 800,
    height = 800,
    yaxis = dict(
      scaleanchor = "x",
      scaleratio = 1,
    )
)
fig.show()